Leiden followed by C means clustering of regulome

First make a proof of concept using igraph

Questions:
- How to measure distance
- How to measure membership
- How to treat small clusters - leave and expand or remove and add nodes to new?

If it works:
- make faster implementation


Problems:
- current distance measure: 
    - favors large clusters and nodes with high degree
    - maybe make measure of distance based on the number of nodes in the cluster
        - in a cluster of 10, 10 edges would make highly conncted to cluster, but in cluster of 100, 10 edges might not.
        - ideally, if node connected to all nodes, with 1 PPV - distance should be nearing 0, if no connection should be 1
        - 1 - (n_nodes_cluster/sum(distances))
            - this will favor smaller clusters

In [1]:
# Import packages

import igraph as ig
import pandas as pd
import numpy as np
import seaborn as sns
from compress_pickle import load, dump

def open_pickle(file):
    with open(path+file, 'rb') as pickle_file:
        return load(path=pickle_file, compression='infer')

path = './files/'

In [3]:
# load regulome
regulome_network_edges = pd.read_csv(filepath_or_buffer=path+'/human_regulome_pd.gz', compression='infer')
regulome_network_edges = regulome_network_edges.astype({'ProteinAid': 'str', 'ProteinBid':'str'})

regulome_graph = ig.Graph.DataFrame(regulome_network_edges, directed=False, use_vids=False)
msigdb_c3_tft_dict = open_pickle('msigdb_c3_tft_dict.pkl')
proteins = pd.concat([regulome_network_edges['ProteinAid'], regulome_network_edges['ProteinBid']]).unique()

del regulome_network_edges

In [ ]:
def leiden_clustering(graph, res, b, n_iter):
    return graph.community_leiden(objective_function='modularity',
                            weights='PPV',
                            resolution=res,
                            beta=b,
                            n_iterations=n_iter)

leiden_clusters = leiden_clustering(regulome_graph, 4, 0.1, 10)
clusters_dict = {n:set(regulome_graph.vs[node]['name'] for node in cluster) for (n, cluster) in enumerate(leiden_clusters)}

In [ ]:
fig = sns.histplot(leiden_clusters.sizes())
fig.set_title("Cluster size distribution of Leiden clusters")
fig.set_xlabel("Cluster Size")
fig.savefig("./images/LiedenClustersDistr.png")

In [60]:

from itertools import chain
from alive_progress import alive_bar
from joblib import Parallel, delayed 

def calc_dist(neighbors, clusters, distance_measure='dist_sum'): # add weight formula argument:
    node_distances = dict()

    for (cluster_id, cluster) in clusters.items():
        connected_nodes = {node : weight for (node, weight) in neighbors.items() if node in cluster}
        # print(connected_nodes)

        if len(connected_nodes) == 0: # no need if no connections
            node_distances[cluster_id] = 1
            continue

        # get weights of each connection to nodes in the cluster
        weights_sum = sum(connected_nodes.values())

        if weights_sum == 0: # dont need this myself, but a thing to look at if want to ship out later
            node_distances[cluster_id] = 1
            continue
        
        match distance_measure:
            case 'dist_sum':
                node_distances[cluster_id] = min(1, (1 / weights_sum)) # set 1 as max distance, should I set other to more ?
            case _:
                return # quit function if wrong function name
    return node_distances


def calc_membership(node_distances, m):
    node_memberships = dict()

    for (cluster_id, d) in node_distances.items():         
        fuzzy_exponent = 2 / (m - 1)

        # give better name - denomintation or membership function  
        membership_den = [(d / other_d) ** fuzzy_exponent for other_d in node_distances.values()]

        node_memberships[cluster_id] = 1 / sum(membership_den)

    return node_memberships


def optimization_function(node_distances, node_memberships, m): # change function name
    node_J = 0
    for cluster_id in node_distances.keys():
        node_J += node_memberships[cluster_id]**m * node_distances[cluster_id]**2
    return node_J
    
def node_iteration(node, neighbors, clusters, m, optimize):
    node_distances = calc_dist(neighbors, clusters) # returns dictionary with distance to each cluster
    node_memberships = calc_membership(node_distances, m) # adds memberships of node to dictionary
    if optimize:
        Ji = optimization_function(node_distances, node_memberships, m)
        return node, node_distances, node_memberships, Ji
    return node, node_distances, node_memberships


def gather_neighbors(graph, nodes, distance_measure='dist_sum'):
    match distance_measure:
        case 'dist_sum':
            return {
                node : {
                    graph.vs[neighbor]['name'] : graph.es[graph.get_eid(node, neighbor)]['PPV'] for neighbor in graph.neighbors(node)
                    } 
                    for node in nodes
                }
        case 'path_len':
            return 


def network_c_means(graph, clusters, m, optimize=False, distance_measure='dist_sum'):
    # add leiden here after 
    nodes = [node['name'] for node in graph.vs]


    memberships_dict = dict()
    distances_dict = dict()
    if optimize:
        J = 0
    results = Parallel(n_jobs=6)(delayed(node_iteration)(node, neighbor_edges_dict[node], clusters, m, optimize) for node in nodes)

    return results

# results = network_c_means(regulome_graph, clusters_dict, 0.5, optimize=True)

In [5]:
from compress_pickle import load
nodes = [node['name'] for node in regulome_graph.vs]

with open(path+'path_lengths.gz', 'rb') as file:
    path_lengths = load(file, compression='infer')


In [ ]:
distances_dict[nodes[-2]]

: 

In [ ]:
from IPython.display import clear_output

nodes = [node for node in regulome_graph.vs['name']]

i = 0
distances_dict = dict()
n_lengths = len(path_lengths[0])
n_nodes = len(nodes)
while i != n_lengths:
    for (node_number, node) in enumerate(nodes):
        clear_output()
        print(f'{node_number}/{n_nodes}')
        distances = dict()
        for other_node in nodes:
            distances[other_node] = path_lengths[0][i]
            if i == n_lengths:
                break
            i += 1
        distances_dict[node] = distances
    
    


15040/15041


KeyError: 226231680

In [ ]:

for i in path_lengths[0]:
    t+=i
    if t == 10:
        break

1.0
1.0
0.9760000109672546
1.7020000219345093
0.9700000286102295
1.7209999561309814
1.0
1.7000000476837158
1.7120000123977661
0.8510000109672546


In [91]:
def calc_distances(graph, node, nodes):
    distances = graph.distances(node, weights='PPV', algorithm='Djikstra')
    distances_dict = {
        other_node : distance
        for (other_node, distance) in zip(nodes, distances)
        if other_node != node
    }

    return distances_dict

def calc_path_lengths(graph):
    nodes = [node for node in graph.vs['name']]
    results = Parallel(n_jobs=6)(delayed(calc_distances)(graph, node, nodes) for node in nodes)

    path_lengths = {
        node : result for (node, result) in zip(nodes, results)
    }

    return path_lengths

path_lengths = calc_path_lengths(regulome_graph)

/tmp/ipykernel_398/318358562.py:2: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
/tmp/ipykernel_398/318358562.py:2: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
/tmp/ipykernel_398/318358562.py:2: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
/tmp/ipykernel_398/318358562.py:2: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
/tmp/ipykernel_398/318358562.py:2: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
/tmp/ipykernel_398/318358562.py:2: DeprecationWarning: Partial string match

KeyboardInterrupt: 

In [51]:
t = regulome_graph.distances('350619', '349606', weights='PPV', algorithm='Djikstra')

n1 = set(regulome_graph.neighbors('350619'))
n2 = set(regulome_graph.neighbors('349606'))

n1.intersection(n2) #{28, 110}

regulome_graph.distances('350619',  weights='PPV')


/tmp/ipykernel_398/57742233.py:1: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
  t = regulome_graph.distances('350619', '349606', weights='PPV', algorithm='Djikstra')


[[1.7,
  1.814,
  1.826,
  1.766,
  1.842,
  1.831,
  1.825,
  1.826,
  1.823,
  2.55,
  1.797,
  1.759,
  1.819,
  1.802,
  1.8199999999999998,
  1.819,
  1.8239999999999998,
  1.764,
  1.803,
  1.799,
  1.8199999999999998,
  1.8439999999999999,
  1.782,
  1.803,
  1.8199999999999998,
  1.838,
  1.847,
  1.821,
  0.85,
  1.706,
  1.756,
  1.821,
  1.8239999999999998,
  1.821,
  1.807,
  1.817,
  1.8119999999999998,
  1.784,
  1.823,
  1.756,
  1.783,
  1.83,
  1.8399999999999999,
  1.811,
  1.79,
  1.801,
  1.845,
  1.744,
  1.821,
  1.7570000000000001,
  1.8119999999999998,
  1.8239999999999998,
  1.749,
  1.814,
  1.791,
  1.835,
  1.83,
  1.7810000000000001,
  1.826,
  1.702,
  1.791,
  1.823,
  1.8199999999999998,
  1.797,
  1.794,
  1.767,
  1.81,
  1.751,
  1.779,
  1.7,
  1.834,
  1.8279999999999998,
  1.8319999999999999,
  1.839,
  1.724,
  1.85,
  1.813,
  1.75,
  1.825,
  1.817,
  1.837,
  1.8119999999999998,
  1.8079999999999998,
  1.842,
  1.838,
  1.786,
  1.7309999999999

In [ ]:
from math import log10

protein_edges = pd.concat([regulome_network_edges['ProteinAid'], regulome_network_edges['ProteinBid']]).value_counts()
most_connected_proteins = tuple(protein_edges.index[:5])
least_connected_proteins = tuple(protein_edges[protein_edges > 5].index[-5:])

def network_c_means_connections(graph, to_investigate, clusters, m, optimize=False):
    # add leiden here after 
    nodes = [node['name'] for node in graph.vs]
    neighbor_edges_dict = {
        node : {
            graph.vs[neighbor]['name'] : graph.es[graph.get_eid(node, neighbor)]['PPV'] for neighbor in graph.neighbors(node)
            } 
            for node in nodes
        }
    results = Parallel(n_jobs=6)(delayed(node_iteration)(node, neighbor_edges_dict[node], clusters, m, optimize) for node in to_investigate)

    memberships_dict = {result[0]: tuple(log10(r) for r in result[2].values()) for result in results}
    return memberships_dict

m_values = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

most_connected_memberships = dict()
for m in m_values:
    most_connected_memberships[m] = network_c_means_connections(regulome_graph, most_connected_proteins, clusters_dict, m)


least_connected_memberships = dict()
for m in m_values:
    least_connected_memberships[m] = network_c_means_connections(regulome_graph, least_connected_proteins, clusters_dict, m)



In [ ]:
# most_connected_memberships

# Manually extract the keys and values
rows = [
    (m, protein, membership)
    for (m, proteins) in most_connected_memberships.items()
    for (protein, memberships) in proteins.items()
    for membership in memberships
]

most_connected_df = pd.DataFrame(rows, columns=['fuzzy_m', 'Proteinid', 'membership'])

fig = sns.FacetGrid(most_connected_df, col='fuzzy_m', row='Proteinid')
fig.map(sns.histplot, 'membership')
fig.figure.subplots_adjust(top=0.925)
fig.set_xlabels('Membership (log10)')
fig.figure.suptitle("Membership distribution of the 5 proteins with the highest degree")
fig.savefig("./images/most_well_connected_sum_dist.png")

In [ ]:
pd.Series(least_connected_memberships[0.5]['331763']).value_counts()

In [ ]:
sns.histplot(least_connected_memberships[0.5]['332631'])

In [ ]:
# least_connected_memberships

# Manually extract the keys and values
rows = [
    (m, protein, membership)
    for (m, proteins) in least_connected_memberships.items()
    for (protein, memberships) in proteins.items()
    for membership in memberships
]

least_connected_df = pd.DataFrame(rows, columns=['fuzzy_m', 'Proteinid', 'membership'])

fig = sns.FacetGrid(least_connected_df, col='fuzzy_m', row='Proteinid')
fig.map(sns.histplot, 'membership')
fig.figure.subplots_adjust(top=0.925)
fig.set_xlabels('Membership (log10)')
fig.figure.suptitle("Membership distribution of the 5 proteins with the lowest degree (more than 5)")

fig.savefig("./images/least_connected_sum_dist_not1.png")

In [ ]:
from itertools import chain

def calc_dist(graph, node, clusters):
    node_distances = dict()
    neighbors = set(graph.neighbors(node))

    for (cluster_id, cluster) in clusters.items():
        connected_nodes = set(neighbors).intersection(cluster)

        if len(connected_nodes) == 0: # no need if no connections
            node_distances[cluster_id] = 1
            continue

        # get weights of each connection to nodes in the cluster
        weights = [graph.es(graph.get_eid(node, connected_node))['PPV'] for connected_node in connected_nodes]
        weights_sum = sum(chain.from_iterable(weights))

        if weights_sum == 0: # dont need this myself, but a thing to look at if want to ship out later
            node_distances[cluster_id] = 1
            continue

        node_distances[cluster_id] = 1 / weights_sum
    return node_distances


def calc_membership(node_distances, m):
    node_memberships = dict()

    for (cluster_id, d) in node_distances.items():         
        fuzzy_exponent = 2 / (m - 1)

        # give better name - denomintation or membership function  
        membership_den = [(d / other_d) ** fuzzy_exponent for other_d in node_distances.values()]

        node_memberships[cluster_id] = 1 / sum(membership_den)

    return node_memberships


def optimization_function(node_distances, node_memberships, m): # change function name
    node_J = 0
    for cluster_id in node_distances.keys():
        node_J += node_memberships[cluster_id]**m * node_distances[cluster_id]**2
    return node_J
    

def network_c_means(graph, clusters, m, optimize=False):
    # add leiden here after 

    membership_dict = dict()
    distances_dict = dict()
    
    if optimize:
        Js = list()

    for i in range(0, 10):
        for node in graph.vs:
            node = node['name']
            node_distances = calc_dist(graph, node, clusters) # returns dictionary with distance to each cluster
            node_memberships = calc_membership(node_distances, m) # adds memberships of node to dictionary

            if optimize:
                Js.append(optimization_function(node_distances, node_memberships, m))
            distances_dict[node] = node_distances # maybe just add directly to dictionary
            membership_dict[node] = node_memberships

    

    return Js

memberships = network_c_means(regulome_graph, clusters_dict, 0.5, optimize=True)